In [4]:
'''
This notebook reads an mp4 video file and extracts all the individual frames.
'''
import sys
sys.path.append('../')
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.metrics import structural_similarity as ssim
from scripts.generate_from_vid import generate

def interpolate_video(path, model, thresh=[0.65, 0.97], normalize=True, double_fps=False):
    new_frames = []
    # Read in the video
    vidcap = cv2.VideoCapture(path)
    print('Video has', int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)), 'frames')
    success, prev_frame = vidcap.read()
    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    if normalize:
        prev_frame = prev_frame / np.max(prev_frame)
        prev_frame[prev_frame < 0.75] = 0
        prev_frame[prev_frame >= 0.75] = 1
    unique_frame = prev_frame.copy()
    target_frame = prev_frame.copy()
    i = 0
    num_unique_frames = 0
    num_duplicate_frames = 0
    num_shot_boundaries = 0
    success = True
    generated_frame = None
    while success:
        success,curr_frame = vidcap.read()
        # Display the read image, not save, just display
        if success:
            curr_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            curr_frame = curr_frame / np.max(curr_frame)
            curr_frame[curr_frame < 0.75] = 0
            curr_frame[curr_frame >= 0.75] = 1
            ssim_diff = ssim(prev_frame, curr_frame, data_range=curr_frame.max() - curr_frame.min(),
                             channel_axis=None)
            unique = thresh[0] <= ssim_diff <= thresh[1]
            duplicate = ssim_diff > thresh[1]
            shot_boundary = ssim_diff < thresh[0]
            if unique:
              unique_frame = prev_frame.copy()
              num_unique_frames += 1
              target_frame = unique_frame.copy()
            if duplicate:
              num_duplicate_frames += 1
              if not double_fps:
                target_frame = generate(model, new_frames[-1], curr_frame)
            if shot_boundary:
              target_frame = prev_frame.copy()
              num_shot_boundaries += 1
            if i == 1:
              target_frame = prev_frame 
            if double_fps and not shot_boundary:
               generated_frame = generate(model, prev_frame, curr_frame)   
        new_frames.append(target_frame)
        if generated_frame is not None:
            new_frames.append(generated_frame)
        prev_frame = curr_frame
        print(i)
        i += 1
    print('Video has', num_unique_frames, 'unique frames')
    print('Video has', num_duplicate_frames, 'duplicate frames')
    print('Video has', num_shot_boundaries, 'shot boundaries')
    print('Interpolated video has', len(new_frames), 'frames')
    return new_frames

In [5]:
from generators.generator_padded import UNetPadded
import torch

thresh = [0.65, 0.95]
model = UNetPadded(2, output_channels=1, hidden_channels=64)
checkpoint_path = '/data/farriaga/Experiments/exp_paddedd/gen_checkpoint.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
new_frames = interpolate_video('test - Trim.mp4', model=model, thresh=thresh,
                               double_fps=True, normalize=True)
print('Extracted', len(new_frames), 'frames')

Using UNetPadded


FileNotFoundError: [Errno 2] No such file or directory: '/data/farriaga/Experiments/exp_paddedd/gen_checkpoint.pth'

In [27]:
path = '/data/farriaga/Experiments/video_results'
for i, frame in enumerate(new_frames):
    cv2.imwrite(f'{path}/frame_{i}.png', frame)

In [10]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 48
output_video_path = '/data/farriaga/Experiments/video/test_double_fps.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (512, 288))
for frame in new_frames:
    frame = (frame * 255).astype(np.uint8)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    out.write(frame)
out.release()

In [5]:
import imageio

import imageio

def video_to_gif(video_path, gif_path):
    # Read the video file
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']  # Get the frames per second
    duration = 1.0 / fps  # Calculate the duration of each frame

    # Write the output GIF file
    with imageio.get_writer(gif_path, mode='I', duration=duration) as writer:
        for frame in video:
            writer.append_data(frame)


In [6]:
video_to_gif('/data/farriaga/Experiments/video/test_double_fps.mp4', 
             '/data/farriaga/Experiments/video/test_double_fps.gif')

In [3]:
from line_extractor.extract import extract_from_video
from generators.generator_padded import UNetPadded
import torch

model = UNetPadded(2, output_channels=1, hidden_channels=64)
checkpoint_path = '/data/farriaga/Experiments/exp_paddedd/gen_checkpoint.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

# extract_from_video('jigokuraku.mp4', model)

ModuleNotFoundError: No module named 'model'